In [1]:
import pandas as pd #import library
import numpy as np
import warnings
warnings.filterwarnings(action='once')

In [2]:
csv = pd.read_csv("rmm_1979.csv") #load csv file of rmm
indf  = pd.DataFrame(csv)         #make it into dataframe

# indf = indf[(indf["month"]>=11) | (indf["month"]<=3) | ((indf["month"] == 4) & (indf["day"]<=1))]

#split into NDJFM

indf.to_csv("indf.csv")

In [3]:
yyyymmdd = pd.to_datetime(indf[["year","month","day"]])    #make yyyymmdd

yyyymmdd = yyyymmdd.dt.strftime('%Y%m%d').astype(int)

yyyymmdd

0        19790101
1        19790102
2        19790103
3        19790104
4        19790105
           ...   
16077    20230107
16078    20230108
16079    20230109
16080    20230110
16081    20230111
Length: 16082, dtype: int64

In [4]:
indf.insert(0,"yyyymmdd", yyyymmdd)            #insert the yyyymmdd to the dataframe

new_df = indf[["yyyymmdd","phase","amp","rmm1","rmm2"]]          #subset the dataframe

# df = new_df[pd.to_datetime(new_df['yyyymmdd']).dt.month == 5]
# df

df = new_df
df

,yyyymmdd,phase,amp,rmm1,rmm2
0,19790101,6,1.060090,0.142507,1.050470
1,19790102,7,1.388700,-0.204200,1.373610
2,19790103,7,1.547580,-0.158610,1.539430
3,19790104,7,1.471080,-0.182450,1.459720
4,19790105,7,1.181000,-0.320050,1.136800
...,...,...,...,...,...
16077,20230107,7,0.674394,-0.162030,0.654640
16078,20230108,7,0.872342,-0.338392,0.804034
16079,20230109,7,0.913857,-0.388525,0.827154
16080,20230110,7,0.746625,-0.287511,0.689048


In [5]:
df_p2_all = df[df["phase"] == 2]   #get phase 2 only
df_p2_all

df_p2     = df_p2_all[df_p2_all["amp"]>=1.0].index.values   #get only active mjo


In [6]:


ind_df = []  #list for the dates

for i in df_p2:      #loop for every phase 2 of the MJO
#     p = df["phase"].iloc[i]     #Get the index of the event
    
    dt = 0
    while dt <= 100 : #placeholder, can be anything high enough
        
        print(df["yyyymmdd"].iloc[i])
        pex  = df["phase"].iloc[i+dt]      #get phase at t = i+dt
        pex2 = df["phase"].iloc[i+dt+1]    #get phase at t = i+dt+1
            
        temp = []                          #make empty list
        if pex == 2 and pex2 == 2:         #if phase t+dt equal to 2 and t+dt+1 equal to 2, stop the loop.
            break
        elif (pex != pex2) or pex>5:       #else if phase t+dt not equal to t+dt+1 or phase t+dt > 5, go to the next cond
            if (pex2 != pex+1) or pex2 < pex:  #if phase t+dt+1 not equal to (phase t+dt)+1 or phase t+dt+1 < phase t+dt, break the loop
                break
        else:
            t = 0 #else, go for the next loop
            
            while t <= dt: #for every time, until last dt, insert the dates into the temp list
                temp.append(df["yyyymmdd"].iloc[i+t])
                t += 1
        
            
            if (df["phase"].iloc[i+dt] == 5)  : #if the last time is phase 5, insert it into ind_df
                ind_df.append(temp)
        dt += 1


print(ind_df)

    

19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790208
19790325
19790326
19790327
19790430
19790501
19790501
19790501
19790501
19790501
19790501
19790501
19790501
19790501
19790501
19790501
19790501
19790501
19790501
19790501
19790607
19790608
19790609
19790610
19790611
19790612
19790613
19790614
19790614
19790614
19790614
19790614
19790614
19790614
19790614
19790614
19790614
19790614
19790908
19790909
19790912
19790916
19790917
19790918
19790918
19790918
19790918
19790918
19790918
19790918
19790918
19790918
19790918
19790918
19790918
19790918
19790918
19790918
19790918
19790918
19790918
19790918
19791031
19791101
19791102
19791103
19791104
19791104
19791104
19791104
19791112
19791113
19791114
1

In [7]:
df_ind = pd.DataFrame(ind_df) #make dataframe
df_ind = df_ind.drop_duplicates(subset = 0, keep="last") #drop duplicates, keep the last iteration of the data

# df_ind = df_ind.astype(int).transpose()
df_ind = df_ind.fillna(19700501).astype(int).transpose() #fill NA with 19700501 (will be discarded anyway), make into int, and transpose

df_ind

#split into NDJFM, but add also early april if the event starts from march
for i in df_ind:
    df_ind[i] = pd.to_datetime(df_ind[i], format='%Y%m%d')
    df_ind[i] = df_ind[i][((df_ind[i].dt.month >= 11) | (df_ind[i].dt.month <= 3)) | 
#                           ((df_ind[i].dt.month == 10) & (df_ind[i].dt.day >= 29)) |
                          ((df_ind[i].dt.month == 4) & (df_ind[i].dt.day <= 10))
                         ]

df_ind = df_ind.dropna(how = "all", axis = 1)

#removing the event that starts from april
df_ind.columns = df_ind.iloc[0]

# print(pd.DatetimeIndex(df_ind.iloc[0]).month )

for i in df_ind.iloc[0]:
#     print(i)
    if i.month == 4:
        df_ind[i] = pd.NaT

df_ind.columns = df_ind.iloc[0]

df_ind    
# print(df_ind)
df_ind = df_ind[[col for col in df_ind.columns if col is not pd.NaT]]
df_ind

#clean the dataframe
c = 1
for col in df_ind.columns:
    
    
    df_ind[col] = df_ind[col].dt.strftime('%Y%m%d').astype("Int32") #change again to yyyymmdd
    
    df_ind.rename(columns={col:"event-"+str(c)},inplace=True) #rename columns
    c+=1

    
df_ind = df_ind.fillna(-9999).astype(int)
df_ind

,event-1,event-2,event-3,event-4,event-5,event-6,event-7,event-8,event-9,event-10,...,event-58,event-59,event-60,event-61,event-62,event-63,event-64,event-65,event-66,event-67
0,19790208,19791116,19800118,19810326,19820107,19821103,19841128,19850203,19851105,19860102,...,20140330,20141119,20170125,20180113,20181105,20181206,20191214,20200320,20201120,20210324
1,19790209,19791117,19800119,19810327,19820108,19821104,19841129,19850204,19851106,19860103,...,20140331,20141120,20170126,20180114,20181106,20181207,20191215,20200321,20201121,20210325
2,19790210,19791118,19800120,19810328,19820109,19821105,19841130,19850205,19851107,19860104,...,20140401,20141121,20170127,20180115,20181107,20181208,20191216,20200322,20201122,20210326
3,19790211,19791119,19800121,19810329,19820110,19821106,19841201,19850206,19851108,19860105,...,20140402,20141122,20170128,20180116,20181108,20181209,20191217,20200323,20201123,20210327
4,19790212,19791120,19800122,19810330,19820111,19821107,19841202,19850207,19851109,19860106,...,20140403,20141123,20170129,20180117,20181109,20181210,20191218,20200324,20201124,20210328
5,19790213,19791121,19800123,19810331,19820112,19821108,19841203,19850208,19851110,19860107,...,20140404,20141124,20170130,20180118,20181110,20181211,20191219,20200325,20201125,20210329
6,19790214,19791122,19800124,19810401,19820113,19821109,19841204,19850209,19851111,19860108,...,20140405,20141125,20170131,20180119,20181111,20181212,20191220,20200326,20201126,20210330
7,19790215,19791123,19800125,19810402,19820114,-9999,19841205,19850210,19851112,19860109,...,20140406,20141126,20170201,20180120,20181112,20181213,20191221,20200327,20201127,20210331
8,19790216,19791124,19800126,19810403,19820115,-9999,19841206,19850211,19851113,19860110,...,20140407,20141127,20170202,20180121,20181113,20181214,-9999,20200328,20201128,20210401
9,19790217,19791125,19800127,19810404,19820116,-9999,19841207,19850212,19851114,19860111,...,20140408,20141128,20170203,20180122,20181114,20181215,-9999,20200329,20201129,20210402


In [8]:
df_ind.to_csv("MJO_sequence_2345.csv")

In [9]:
#CHECK with CES index
#load data

ces_data = pd.read_csv("CES_dates.txt", header=None)
df_ces   = pd.DataFrame(ces_data)
df_ces.rename(columns={0:"dates"})


,dates
0,19790124
1,19790125
2,19790208
3,19790303
4,19791212
...,...
467,20200219
468,20200220
469,20200221
470,20200224


In [10]:
#checking

contained = []
for i in df_ind:
    temp = np.in1d(df_ind[i],df_ces, invert=False) #check data if any CES coincided with MJO DAYS
    contained.append(temp)

df_contained = pd.DataFrame(contained) #TURN into dataframe

df_contained.replace({False: 0, True: 1}, inplace=True) #replace true/false into 1 to 0 
df_contained = df_contained.transpose()

df_contained.columns = df_ind.columns
df_contained


    

,event-1,event-2,event-3,event-4,event-5,event-6,event-7,event-8,event-9,event-10,...,event-58,event-59,event-60,event-61,event-62,event-63,event-64,event-65,event-66,event-67
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [11]:
#make a nice dataframe with yyyymmdd1,flag1,yyyymmdd2,flag2,etc

df_end = pd.DataFrame()

colnum = len(df_ind.columns)
print(colnum)

x=1
while x <= colnum:
    df_end["event-"+str(x)] = df_ind["event-"+str(x)]
    df_end["flag-"+str(x)]   = df_contained["event-"+str(x)]
    df_end["flag_simple-"+str(x)] = df_contained["event-"+str(x)].any()*1
    x = x+1

# colnum
df_end

    

67


/var/folders/41/txq0cr4j1kl0vqjl56x7l60w0000gp/T/ipykernel_33241/1504335575.py:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_end["flag-"+str(x)]   = df_contained["event-"+str(x)]
/var/folders/41/txq0cr4j1kl0vqjl56x7l60w0000gp/T/ipykernel_33241/1504335575.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_end["flag_simple-"+str(x)] = df_contained["event-"+str(x)].any()*1
/var/folders/41/txq0cr4j1kl0vqjl56x7l60w0000gp/T/ipykernel_33241/1504335575.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usual

,event-1,flag-1,flag_simple-1,event-2,flag-2,flag_simple-2,event-3,flag-3,flag_simple-3,event-4,...,flag_simple-64,event-65,flag-65,flag_simple-65,event-66,flag-66,flag_simple-66,event-67,flag-67,flag_simple-67
0,19790208,1,1,19791116,0,0,19800118,0,0,19810326,...,0,20200320,0,0,20201120,0,1,20210324,0,0
1,19790209,0,1,19791117,0,0,19800119,0,0,19810327,...,0,20200321,0,0,20201121,0,1,20210325,0,0
2,19790210,0,1,19791118,0,0,19800120,0,0,19810328,...,0,20200322,0,0,20201122,0,1,20210326,0,0
3,19790211,0,1,19791119,0,0,19800121,0,0,19810329,...,0,20200323,0,0,20201123,0,1,20210327,0,0
4,19790212,0,1,19791120,0,0,19800122,0,0,19810330,...,0,20200324,0,0,20201124,0,1,20210328,0,0
5,19790213,0,1,19791121,0,0,19800123,0,0,19810331,...,0,20200325,0,0,20201125,0,1,20210329,0,0
6,19790214,0,1,19791122,0,0,19800124,0,0,19810401,...,0,20200326,0,0,20201126,0,1,20210330,0,0
7,19790215,0,1,19791123,0,0,19800125,0,0,19810402,...,0,20200327,0,0,20201127,0,1,20210331,0,0
8,19790216,0,1,19791124,0,0,19800126,0,0,19810403,...,0,20200328,0,0,20201128,0,1,20210401,0,0
9,19790217,0,1,19791125,0,0,19800127,0,0,19810404,...,0,20200329,0,0,20201129,0,1,20210402,0,0


In [12]:
df_end.to_csv("MJOsequence_2345_ces_flag.csv")

In [13]:
#pisah tanggal MJO sequence dgn CES event dan non-event 

event = []
non_event = []

num = len(df_ind)
numevent = len(df_ind.columns)
print(num)
print(numevent)
sequence = []
sequence_non = []

counter = 1

j = 1
counter = 1
counter_non = 1
while j < numevent:
#     print(counter)
    i = 0
    counter = counter+1
    counter_non = counter_non+1
    while i <= num-1:
        if df_end["flag_simple-"+str(j+1)].iloc[i] == 1 and df_end["event-"+str(j+1)].iloc[i] != -9999:
            event.append(df_end["event-"+str(j+1)].iloc[i])
            sequence.append(counter)
            
#             df_end["sequence"] = counter
        elif df_end["flag_simple-"+str(j+1)].iloc[i] == 0 and df_end["event-"+str(j+1)].iloc[i] != -9999:
            non_event.append(df_end["event-"+str(j+1)].iloc[i])
            sequence_non.append(counter_non)
        i = i+1 
    j = j+1

# print(len(seq))
    
# print(df_end)
print(sequence)

38
67
[10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 22, 22, 22, 22, 22, 22, 22, 22, 22, 22, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 35, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 41, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 42, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 43, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 55, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 56, 5

In [14]:

#give each event a unique IDs
from collections import defaultdict

temp = defaultdict(lambda: len(temp))
seqnumber_old = [temp[ele] for ele in sequence]

seqnumber = [x+1 for x in seqnumber_old] #+1, bcs strt from 0
seqnumber



[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 

In [15]:
temp = defaultdict(lambda: len(temp))
seqnumber_non_old = [temp[ele] for ele in sequence_non]

seqnumber_non = [x+1 for x in seqnumber_non_old] #+1, bcs strt from 0
seqnumber_non

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 10,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 11,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 14,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 15,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 16,
 1

In [16]:
# event
# print(seq)
event_df = pd.DataFrame(event)
event_df["seqnumber"] = seqnumber
# event_df = event_df.replace(-9999, None).dropna()
print(event_df)
event_df.to_csv("MJO_CES_event_method1.csv")

            0  seqnumber
0    19860102          1
1    19860103          1
2    19860104          1
3    19860105          1
4    19860106          1
..        ...        ...
300  20201214         18
301  20201215         18
302  20201216         18
303  20201217         18
304  20201218         18

[305 rows x 2 columns]


In [17]:

non_event_df = pd.DataFrame(non_event)
non_event_df["seqnumber"] = seqnumber_non

print(non_event_df)
non_event_df.to_csv("MJO_CES_non_event_method1.csv")


            0  seqnumber
0    19791116          1
1    19791117          1
2    19791118          1
3    19791119          1
4    19791120          1
..        ...        ...
686  20210331         48
687  20210401         48
688  20210402         48
689  20210403         48
690  20210404         48

[691 rows x 2 columns]


In [19]:
#NON_EVENT

mean_df = event_df.copy()

event_df["phase"] = None
event_df["amp"]   = None
event_df["rmm1"]  = None
event_df["rmm2"]  = None

for x in event_df.columns:
    for i in event_df[x].iloc[:]:
#         print(i)
        temp = df[df["yyyymmdd"] == i]
#         print(x)
        idx = event_df.loc[event_df[0] == i].index
#         print(idx)
        event_df["phase"].iloc[idx.values] = temp["phase"].values
        event_df["amp"].iloc[idx.values] = temp["amp"].values
        event_df["rmm1"].iloc[idx.values] = temp["rmm1"].values
        event_df["rmm2"].iloc[idx.values] = temp["rmm2"].values

event_df

mean_p2_amp = event_df.loc[event_df["phase"] == 2, 'amp'].mean()
mean_p3_amp = event_df.loc[event_df["phase"] == 3, 'amp'].mean()
mean_p4_amp = event_df.loc[event_df["phase"] == 4, 'amp'].mean()
mean_p5_amp = event_df.loc[event_df["phase"] == 5, 'amp'].mean()

mean_p2_rmm1 = event_df.loc[event_df["phase"] == 2, 'rmm1'].mean()
mean_p3_rmm1 = event_df.loc[event_df["phase"] == 3, 'rmm1'].mean()
mean_p4_rmm1 = event_df.loc[event_df["phase"] == 4, 'rmm1'].mean()
mean_p5_rmm1 = event_df.loc[event_df["phase"] == 5, 'rmm1'].mean()

mean_p2_rmm2 = event_df.loc[event_df["phase"] == 2, 'rmm2'].mean()
mean_p3_rmm2 = event_df.loc[event_df["phase"] == 3, 'rmm2'].mean()
mean_p4_rmm2 = event_df.loc[event_df["phase"] == 4, 'rmm2'].mean()
mean_p5_rmm2 = event_df.loc[event_df["phase"] == 5, 'rmm2'].mean()

mean_p2_df = pd.DataFrame([20200101, 0, 2, mean_p2_amp,mean_p2_rmm1,mean_p2_rmm2]).T
mean_p3_df = pd.DataFrame([20200101, 0, 3, mean_p3_amp,mean_p3_rmm1,mean_p3_rmm2]).T
mean_p4_df = pd.DataFrame([20200101, 0, 4, mean_p4_amp,mean_p4_rmm1,mean_p4_rmm2]).T
mean_p5_df = pd.DataFrame([20200101, 0, 5, mean_p5_amp,mean_p5_rmm1,mean_p5_rmm2]).T

mean_p2_df.columns = [0,"seqnumber","phase","amp","rmm1", "rmm2"]
mean_p3_df.columns = [0,"seqnumber","phase","amp","rmm1", "rmm2"]
mean_p4_df.columns = [0,"seqnumber","phase","amp","rmm1", "rmm2"]
mean_p5_df.columns = [0,"seqnumber","phase","amp","rmm1", "rmm2"]


mean_p2_df = pd.concat([mean_p2_df,mean_p3_df], ignore_index=True)
mean_p2_df = pd.concat([mean_p2_df,mean_p4_df], ignore_index=True)
mean_p2_df = pd.concat([mean_p2_df,mean_p5_df], ignore_index=True)
# mean_p2_df = pd.concat([mean_p2_df,mean_p5_df], ignore_index=True)

# mean_df
mean_p2_df[[0,"phase","seqnumber"]] = mean_p2_df[[0,"phase","seqnumber"]].astype(int)
mean_df

mean_p2_df.to_csv("MJO_CES_event_method1_mean.csv")
mean_p2_df

/var/folders/41/txq0cr4j1kl0vqjl56x7l60w0000gp/T/ipykernel_33241/2091178559.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  event_df["phase"].iloc[idx.values] = temp["phase"].values


,0,seqnumber,phase,amp,rmm1,rmm2
0,20200101,0,2,1.841469,-0.171553,-1.826911
1,20200101,0,3,1.941803,0.700369,-1.764039
2,20200101,0,4,1.757223,1.605406,-0.614443
3,20200101,0,5,1.660767,1.541255,0.522386


In [21]:
#NON_EVENT

non_event_df["phase"] = None
non_event_df["amp"]   = None
non_event_df["rmm1"]  = None
non_event_df["rmm2"]  = None

for x in non_event_df.columns:
    for i in non_event_df[x].iloc[:]:
#         print(i)
        temp = df[df["yyyymmdd"] == i]
#         print(x)
        idx = non_event_df.loc[non_event_df[0] == i].index
#         print(idx)
        non_event_df["phase"].iloc[idx.values] = temp["phase"].values
        non_event_df["amp"].iloc[idx.values] = temp["amp"].values
        non_event_df["rmm1"].iloc[idx.values] = temp["rmm1"].values
        non_event_df["rmm2"].iloc[idx.values] = temp["rmm2"].values

non_event_df

mean_p2_amp = non_event_df.loc[non_event_df["phase"] == 2, 'amp'].mean()
mean_p3_amp = non_event_df.loc[non_event_df["phase"] == 3, 'amp'].mean()
mean_p4_amp = non_event_df.loc[non_event_df["phase"] == 4, 'amp'].mean()
mean_p5_amp = non_event_df.loc[non_event_df["phase"] == 5, 'amp'].mean()

mean_p2_rmm1 = non_event_df.loc[non_event_df["phase"] == 2, 'rmm1'].mean()
mean_p3_rmm1 = non_event_df.loc[non_event_df["phase"] == 3, 'rmm1'].mean()
mean_p4_rmm1 = non_event_df.loc[non_event_df["phase"] == 4, 'rmm1'].mean()
mean_p5_rmm1 = non_event_df.loc[non_event_df["phase"] == 5, 'rmm1'].mean()

mean_p2_rmm2 = non_event_df.loc[non_event_df["phase"] == 2, 'rmm2'].mean()
mean_p3_rmm2 = non_event_df.loc[non_event_df["phase"] == 3, 'rmm2'].mean()
mean_p4_rmm2 = non_event_df.loc[non_event_df["phase"] == 4, 'rmm2'].mean()
mean_p5_rmm2 = non_event_df.loc[non_event_df["phase"] == 5, 'rmm2'].mean()

mean_p2_df = pd.DataFrame([20200101, 0, 2, mean_p2_amp,mean_p2_rmm1,mean_p2_rmm2]).T
mean_p3_df = pd.DataFrame([20200101, 0, 3, mean_p3_amp,mean_p3_rmm1,mean_p3_rmm2]).T
mean_p4_df = pd.DataFrame([20200101, 0, 4, mean_p4_amp,mean_p4_rmm1,mean_p4_rmm2]).T
mean_p5_df = pd.DataFrame([20200101, 0, 5, mean_p5_amp,mean_p5_rmm1,mean_p5_rmm2]).T

mean_p2_df.columns = [0,"seqnumber","phase","amp","rmm1", "rmm2"]
mean_p3_df.columns = [0,"seqnumber","phase","amp","rmm1", "rmm2"]
mean_p4_df.columns = [0,"seqnumber","phase","amp","rmm1", "rmm2"]
mean_p5_df.columns = [0,"seqnumber","phase","amp","rmm1", "rmm2"]


mean_p2_df = pd.concat([mean_p2_df,mean_p3_df], ignore_index=True)
mean_p2_df = pd.concat([mean_p2_df,mean_p4_df], ignore_index=True)
mean_p2_df = pd.concat([mean_p2_df,mean_p5_df], ignore_index=True)
# mean_p2_df = pd.concat([mean_p2_df,mean_p5_df], ignore_index=True)

# mean_df
mean_p2_df[[0,"phase","seqnumber"]] = mean_p2_df[[0,"phase","seqnumber"]].astype(int)
mean_df

mean_p2_df.to_csv("MJO_CES_non_event_method1_mean.csv")
mean_p2_df

/var/folders/41/txq0cr4j1kl0vqjl56x7l60w0000gp/T/ipykernel_33241/2663354412.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_event_df["phase"].iloc[idx.values] = temp["phase"].values


,0,seqnumber,phase,amp,rmm1,rmm2
0,20200101,0,2,1.739176,-0.175805,-1.724948
1,20200101,0,3,1.969533,0.773052,-1.758260
2,20200101,0,4,1.831888,1.634749,-0.725127
3,20200101,0,5,1.718385,1.619230,0.483791
